In [ ]:
from configparser import ConfigParser

config = ConfigParser()
config.read('config.ini')
config['augmentations']['pad_trunc_noise_multiplier']


In [ ]:
from torchsummary import summary
summary(model, (1,257, 63))

In [ ]:
import torch
from loader.AudioDataset import specMask, collate_batch, createDataset
from torch.utils.data import Dataset
import loader.utils as utils
import matplotlib.pyplot as plt
import model
import random

random.seed(10)
torch.manual_seed(10)

linux = '/media/jianquan/Data/Processed Audio/'
windows = 'E:/Processed Audio/'

audio_paths, _ = utils.getAudioPaths(windows)

audio_train_dataset = createDataset(audio_paths, transformParams = utils.getTransforms(False), outputAudio = True)

test_dataloader = torch.utils.data.DataLoader(
    audio_train_dataset,
    batch_size=1,
    num_workers=0,
    shuffle=False,
    collate_fn = collate_batch
)
batch = next(iter(test_dataloader))
batch_size = len(batch[0])

nfftList = [64, 128, 256, 512, 1024]
classList = [0, 1, 2]

for nfft in nfftList:
    for specClass in classList:
        saveName = f'nfft{nfft}_class{specClass}'
        print(saveName, end=': ')
        testModel = model.CNNNetwork(nfft, specClass)
        testModel(batch[0])
        plt.savefig(f'./spec/spec/whisper Norm/{saveName}.png')


# import IPython.display as ipd
# ipd.Audio(batch[0][3], rate = 8000)
# for b in batch[0]:
#     # print(b.max(),b.median(),b.mean(), b.min())
#     # print(b).

    
#     plt.plot(b[0])
#     plt.show()

In [28]:
import models.segmentation 
model=models.segmentation.PyanNet(num_channels=1)


In [1]:
import glob

def select_model(path='.'):
    # Use glob to find all available models in the specified path
    models = glob.glob(f'{path}/*.model')

    # Print the available models to the user
    print('Available models:')
    for i, model in enumerate(models):
        print(f'{i+1}: {model}')

    # Prompt the user to select a model
    selection = input('Please select a model by entering its number: ')

    # Return the selected model
    return models[int(selection) - 1]


In [7]:
import os
import importlib, inspect


def select_model():
    model_class = None
    # Get the list of files in the 'model' directory
    files = os.listdir('model')
    # Filter out any files that are not Python files
    files = [file for file in files if file.endswith('.py')]

    # Print a list of available models
    print('Available models:')
    for i, file in enumerate(files):
        print(f'{i+1}. {file[:-3]}')

    # Prompt the user to select a model
    while model_class is None:
        # Get the user's input
        selection = input('Enter the number of the model you want to use: ')
        try:
            # Convert the input to an integer
            selection = int(selection)
            # Check if the selection is within the range of available models
            if selection > 0 and selection <= len(files):
                # Import the selected module
                module = importlib.import_module(
                    'model.' + files[selection - 1][:-3])
                # Loop through the attributes in the module
                for attr in dir(module):
                    # Check if the attribute is a class
                    if inspect.isclass(getattr(module, attr)):
                        # Get the class
                        model_class = getattr(module, attr)
                        break
            else:
                # If the selection is not within the range of available models, print an error message
                print('Invalid selection')
        except ValueError:
            # If the input cannot be converted to an integer, print an error message
            print('Invalid input')

    return model_class


In [9]:
model = select_model()

Available models:
1. CNN
2. __init__
3. CNN_mel
4. CNN_specNorm
